In [1]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

## Options

In [2]:
# parse options
problem = 'twelve_pieces_process.json'
problem_subdir = 'results'

recompute_action_states = False
load_external_movements = False

In [15]:
# generic planning options
id_only = 'A2_M1' # None | 'A273_M0'
free_motion_only = False
low_res = True

In [17]:
# client options
viewer = True
debug = True
verbose = True
diagnosis = True
watch = False
step_sim = False

disable_env = False
reinit_tool = False

## Parse process from json

In [5]:
import os
from termcolor import cprint
import pybullet_planning as pp
from integral_timber_joints.planning.parsing import parse_process, save_process_and_movements, get_process_path, save_process

In [6]:
process = parse_process(problem, subdir=problem_subdir)
result_path = get_process_path(problem, subdir='results')
if len(process.movements) == 0:
    cprint('No movements found in process, trigger recompute actions.', 'red')
    recompute_action_states = True
if recompute_action_states:
    cprint('Recomputing Actions and States', 'cyan')
    recompute_action_states(process)

Process json parsed from c:\users\harry\dropbox (mit)\code_ws_dropbox\itj_ws\integral_timber_joints\external\itj_design_study\210128_RemodelFredPavilion\results\twelve_pieces_process.json


In [7]:
# # force load external if only planning for the free motions
# load_external_movements = load_external_movements or free_motion_only or id_only is not None
# if load_external_movements:
#     ext_movement_path = os.path.dirname(result_path)
#     cprint('Loading external movements from {}'.format(ext_movement_path), 'cyan')
#     process.load_external_movements(ext_movement_path)
#     if recompute_action_states:
#         save_process(process, result_path)
#         cprint('Recomputed process saved to %s' % result_path, 'green')

Loading external movements from c:\users\harry\dropbox (mit)\code_ws_dropbox\itj_ws\integral_timber_joints\external\itj_design_study\210128_RemodelFredPavilion\results


## Start client

In [30]:
from integral_timber_joints.planning.robot_setup import load_RFL_world
from integral_timber_joints.planning.run import set_initial_state

# * Connect to path planning backend and initialize robot parameters
client, robot, _ = load_RFL_world(viewer=viewer or diagnosis or view_states or watch or step_sim, verbose=False)
set_initial_state(client, robot, process, disable_env=disable_env, reinit_tool=reinit_tool)

In [29]:
# if id_only:
#     beam_id = process.get_beam_id_from_movement_id(id_only)
#     process.get_movement_summary_by_beam_id(beam_id)

In [31]:
from integral_timber_joints.planning.robot_setup import GANTRY_ARM_GROUP, GANTRY_GROUP, BARE_ARM_GROUP
from compas.robots import Joint
import numpy as np

joint_names = robot.get_configurable_joint_names(group=GANTRY_ARM_GROUP)
joint_types = robot.get_joint_types_by_names(joint_names)
# 0.1 rad = 5.7 deg
joint_jump_threshold = {jt_name : np.pi/6 \
        if jt_type in [Joint.REVOLUTE, Joint.CONTINUOUS] else 0.1 \
        for jt_name, jt_type in zip(joint_names, joint_types)}

options = {
    'debug' : debug,
    'diagnosis' : diagnosis,
    'distance_threshold' : 0.0012,
    'frame_jump_tolerance' : 0.0012,
    'verbose' : verbose,
#     'use_stored_seed' : args.use_stored_seed,
    'jump_threshold' : joint_jump_threshold,
#     'max_distance' : args.max_distance,
}

In [39]:
from integral_timber_joints.planning.stream import compute_free_movement, compute_linear_movement
from integral_timber_joints.planning.solve import compute_movement

GANTRY_ATTEMPTS = 10

chosen_m = process.get_movement_by_movement_id(id_only)

lm_options = options.copy()
lm_options.update({
    'max_step' : 0.01, # interpolation step size, in meter
    'distance_threshold':0.002, # collision checking tolerance, in meter
    'gantry_attempts' : GANTRY_ATTEMPTS,  # gantry attempt matters more
    'cartesian_attempts' : 5, # boosting up cartesian attempt here does not really help
    'reachable_range' : (0.2, 2.8), # circle radius for sampling gantry base when computing IK
    # -------------------
    'planner_id' : 'IterativeIK',
    'cartesian_move_group' : GANTRY_ARM_GROUP,
    # -------------------
    # 'planner_id' : 'LadderGraph',
    # 'ik_function' : _get_sample_bare_arm_ik_fn(client, robot),
    # 'cartesian_move_group' : BARE_ARM_GROUP,
    })

compute_movement(client, robot, process, chosen_m, options=options, diagnosis=diagnosis)

RoboticLinearMovement(#A2_M1, Linear Advance to Final Frame of Beam ('b0'), traj 1)
end conf FK inconsistent (0.00005 m) with given current frame in end state.
Both start/end confs are pre-specified, problem might be too stiff to be solved.
One-sided Cartesian planning : start conf set, forward mode
	cartesian trial #0
Plan found by IterativeIK! After 0 path failure (by IterativeIK) over 1 samples.


True

In [40]:
from integral_timber_joints.planning.visualization import visualize_movement_trajectory

with pp.WorldSaver():
    visualize_movement_trajectory(client, robot, process, chosen_m, step_sim=True)

===
Viz:
RoboticLinearMovement(#A2_M1, Linear Advance to Final Frame of Beam ('b0'), traj 1)


Step conf. 
Step conf. 
Step conf. 
Step conf. 
Step conf. 
Step conf. 
Step conf. 
End state. 


In [41]:
pp.disconnect()